In [6]:
import cv2
import os
import datetime

haar_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(haar_path)

def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    return faces

def detect_from_image(image_path):
    if not os.path.exists(image_path):
        print("Không tìm thấy ảnh:", image_path)
        return

    image = cv2.imread(image_path)
    faces = detect_faces(image)

    print(f"Phát hiện {len(faces)} khuôn mặt trong ảnh.")

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image, "Face", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("Kết quả nhận diện từ ảnh", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    os.makedirs("resuilt_haar_cascade_imgs", exist_ok=True)
    filename = os.path.basename(image_path)
    output_path = os.path.join("resuilt_haar_cascade_imgs", f"detected_{filename}")

    cv2.imwrite(output_path, image)
    print(f"Ảnh đã được lưu vào: {output_path}")

def detect_from_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Không thể mở webcam.")
        return
    output_dir = "resuilt_haar_cascade_webcam"
    os.makedirs(output_dir, exist_ok=True)
    log_file = open("log.txt", "w", encoding="utf-8")

    latest_face_frame = None
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        faces = detect_faces(frame)

        if len(faces) > 0:
            print(f"[{frame_id}] Phát hiện {len(faces)} khuôn mặt.")
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_file.write(f"[{timestamp}] Frame {frame_id}: {len(faces)} faces detected.\n")

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Face detected", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            latest_face_frame = frame.copy()

        cv2.imshow("Phát hiện khuôn mặt", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('e') or key == 27:
            print("Đã thoát quét webcam.")
            break

    cap.release()
    cv2.destroyAllWindows()
    log_file.close()

    if latest_face_frame is not None:
        output_path = os.path.join(output_dir, "Haar_Cascade_last.jpg")
        cv2.imwrite(output_path, latest_face_frame)
        print(f"Ảnh cuối cùng có khuôn mặt đã được lưu vào: {output_path}")
    else:
        print("Không phát hiện khuôn mặt nào trong suốt phiên làm việc.")

def main():
    while True:
        print("\n=== Chương trình Nhận diện Khuôn mặt bằng Haar Cascade ===")
        print("1. Nhận diện từ Webcam")
        print("2. Nhận diện từ Ảnh")
        print("3. Thoát chương trình")
        choice = input("Nhập lựa chọn (1/2/3): ")

        if choice == '1':
            detect_from_webcam()
        elif choice == '2':
            path = input("Nhập đường dẫn tới ảnh: ")
            detect_from_image(path)
        elif choice == '3':
            print("Đã thoát chương trình.")
            break
        else:
            print("Lựa chọn không hợp lệ.")

if __name__ == "__main__":
    main()



=== Chương trình Nhận diện Khuôn mặt bằng Haar Cascade ===
1. Nhận diện từ Webcam
2. Nhận diện từ Ảnh
3. Thoát chương trình
Đã thoát chương trình.
